## 신경망 모델 구성하기

신경망은 데이터에 대한 연산을 수행하는 계층(layer)/모듈(module)로 구성되어 있습니다.

`torch.nn` 네임스페이스는 신경망을 구성하는데 필요한 모든 구성 요소를 제공합니다. **PyTorch의 모든 모듈은 nn.Module 의 하위 클래스(subclass)** 입니다. 

신경망은 다른 모듈(계층; layer)로 구성된 모듈입니다. 이러한 중첩된 구조는 복잡한 아키텍처를 쉽게 구축하고 관리할 수 있습니다.

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

C:\Users\bini\anaconda3\envs\anaconda\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\bini\anaconda3\envs\anaconda\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
C:\Users\bini\anaconda3\envs\anaconda\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)


## 학습을 위한 장치 얻기

가능한 경우 GPU와 같은 하드웨어 가속기에서 모델을 학습하려고 합니다. torch.cuda 를 사용할 수 있는지 확인하고 그렇지 않으면 CPU를 계속 사용합니다.

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cpu device


## 클래스 정의하기

신경망 모델을 `nn.Module 의 하위클래스로 정의`하고, `__init__` 에서 `신경망 계층들을 초기화`합니다. nn.Module 을 상속받은 모든 클래스는 `forward 메소드`에 입력 데이터에 대한 연산들을 구현합니다.

In [14]:
class NeuralNetwork(nn.Module):
    def __init__(self) :
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10),
            nn.ReLU()
        )
        
    def forward(self, x) :
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

NeuralNetwork 의 `인스턴스(instance)를 생성`하고 이를 `device 로 이동`한 뒤, `구조(structure)를 출력`합니다.

In [15]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


모델을 사용하기 위해 `입력 데이터를 전달`합니다. 이는 일부 백그라운드 연산들 과 함께 `모델의 forward 를 실행`합니다. **model.forward() 를 직접 호출하지 마세요!**

모델에 입력을 호출하면 각 분류(class)에 대한 원시(raw) 예측값이 있는 `10-차원 텐서가 반환`됩니다. 원시 예측값을 `nn.Softmax 모듈`의 인스턴스에 통과시켜 `예측 확률`을 얻습니다.

In [16]:
X = torch.rand(1,28,28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class : {y_pred}")

Predicted class : tensor([6])


---------------------------

## 모델 계층(Layer)

FashionMNIST `모델의 계층들`을 살펴보겠습니다. 이를 설명하기 위해, `28x28 크기의 이미지 3개로 구성된 미니배치`를 가져와, 신경망을 통과할 때 어떤 일이 발생하는지 알아보겠습니다.

In [22]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


### nn.Flatten

nn.Flatten 계층을 초기화하여 각 28x28의 2D 이미지를 784 픽셀 값을 갖는 연속된 배열로 변환합니다. (dim=0의 미니배치 차원은 유지됩니다.)

In [23]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


### nn.Linear

선형 계층 은 저장된 `가중치(weight)`와 `편향(bias)`을 사용하여 입력에 선형 변환(linear transformation)을 적용하는 모듈입니다.

In [24]:
layer1 = nn.Linear(28*28, 20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


### nn.ReLU

`비선형 활성화(activation)`는 모델의 입력과 출력 사이에 복잡한 관계(mapping)를 만듭니다. 비선형 활성화는 선형 변환 후에 적용되어 `비선형성(nonlinearity) 을 도입`하고, 신경망이 다양한 현상을 학습할 수 있도록 돕습니다.

이 모델에서는 nn.ReLU 를 선형 계층들 사이에 사용하지만, 모델을 만들 때는 비선형성을 가진 다른 활성화를 도입할 수도 있습니다.

In [25]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}") #0~무한대 값으로

Before ReLU: tensor([[ 5.6971e-01, -5.7115e-01,  2.8998e-01, -9.0222e-02, -3.1686e-01,
         -1.4616e-01,  1.8606e-01, -1.4556e-01,  2.1503e-01, -4.6697e-01,
          2.5319e-01,  3.9238e-01,  9.1407e-02, -2.6282e-01, -2.8045e-01,
         -1.7538e-01,  2.1283e-01, -4.3222e-01,  2.0004e-01,  3.9292e-01],
        [ 5.6490e-01, -5.5442e-01,  2.9500e-01,  1.0738e-01, -5.5200e-01,
         -3.4168e-01,  1.2851e-01, -4.8399e-02,  2.7103e-01, -2.3012e-01,
          3.3276e-01, -1.8378e-02,  1.4484e-04, -8.0929e-01,  3.9357e-01,
         -8.2259e-02,  2.9916e-01, -1.4648e-01,  6.2712e-02,  2.0307e-01],
        [ 5.2247e-01, -3.8251e-01,  1.0365e-01,  2.0182e-01, -2.9526e-01,
         -2.5334e-01,  5.7528e-03, -2.2325e-01, -2.4618e-02, -4.4360e-01,
          3.3402e-01,  2.6535e-01,  2.7775e-02, -4.4682e-02,  1.2021e-01,
         -3.9599e-01,  6.3906e-01, -4.8033e-01,  2.4258e-02,  5.9568e-01]],
       grad_fn=<AddmmBackward>)


After ReLU: tensor([[5.6971e-01, 0.0000e+00, 2.8998e-01, 0.00

### nn.Sequential

`nn.Sequential` 은 `순서를 갖는 모듈의 컨테이너`입니다. 데이터는 정의된 것과 같은 순서로 모든 모듈들을 통해 전달됩니다. `순차 컨테이너(sequential container)`를 사용하여 아래의 `seq_modules` 와 같은 신경망을 빠르게 만들 수 있습니다.

In [31]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20,10)
)

In [41]:
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)
logits.size()

torch.Size([3, 10])

### nn.Softmax

신경망의 마지막 선형 계층은 nn.Softmax 모듈에 전달될 ([-infty, infty] 범위의 원시 값(raw value)인) logits 를 반환합니다. logits는 모델의 `각 분류(class)에 대한 예측 확률을 나타내도록 [0, 1] 범위로 비례하여 조정(scale)`됩니다. dim 매개변수는 값의 합이 1이 되는 차원을 나타냅니다.

In [44]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)
print(pred_probab.size())
pred_probab.argmax(1)

torch.Size([3, 10])


tensor([9, 3, 3])

## 모델 매개변수

신경망 내부의 많은 계층들은 `매개변수화(parameterize)` 됩니다. 즉, 학습 중에 최적화되는 가중치와 편향과 연관지어집니다. `nn.Module 을 상속`하면 `모델 객체 내부의 모든 필드들이 자동으로 추적(track)`되며, `모델의 parameters()` 및 `named_parameters()` 메소드로 모든 매개변수에 접근할 수 있게 됩니다.

이 예제에서는 각 매개변수들을 순회하며(iterate), 매개변수의 크기와 값을 출력합니다.

In [45]:
print("Model structure: ", model, "\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure:  NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
) 


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0338, -0.0134, -0.0031,  ...,  0.0261, -0.0323,  0.0328],
        [-0.0163,  0.0052,  0.0113,  ...,  0.0071, -0.0299, -0.0285]],
       grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0304, -0.0086], grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0048, -0.0392, -0.0131,  ..., -0.0109, -0.0249,  0.0433],
        [-0.0326,  0.0082, -0.0003,  ..., -0.0408,  0.0010, -0.0231]],
       grad_fn=<SliceBackward>) 

Layer: linear_relu_